In [1]:
import torch

/home/k/k202141/.conda/envs/iconml/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.cuda.is_available()

True

In [3]:
import numpy as np

In [4]:
import os
import numpy as np
import pytorch_lightning as pl
import torch
import torch.nn as nn

class plNetwork(nn.Module):
    def __init__(
        self,
        act,
        n_layers=5,
        ns=200,
        out_features=4,
        depth=9,
        p=0.25,
        use_batch_norm=False,
        use_dropout=False,
    ):

        super().__init__()

        self.layers = []
        self.layer_size = []

        in_features = depth
        for i in range(n_layers):

            self.layers.append(nn.Linear(in_features, ns))
            # nn.init.kaiming_normal_(self.layers[i].weight, mode='fan_out')
            if use_batch_norm:
                self.layers.append(nn.BatchNorm1d(ns))
            in_features = ns

        self.layers.append(nn.Linear(ns, out_features))
        self.layers = nn.ModuleList(self.layers)
        self.dropout = nn.Dropout(p)
        self.activation = act
        self.n_layers = n_layers
        self.use_dropout = use_dropout

    def _forward_impl(self, x):
        for i in range(self.n_layers - 1):
            x = self.activation(self.layers[i](x))
            if self.use_dropout:
                x = self.dropout(x)

        x = self.layers[-1](x)

        return x

    def forward(self, x):
        return self._forward_impl(x)

class LightningModel(pl.LightningModule):
    def __init__(
        self,
        updates_mean,
        updates_std,
        inputs_mean,
        inputs_std,
        save_dir=None,
        batch_size=256,
        beta=0.35,
        learning_rate=2e-4,
        act=nn.ReLU(),
        loss_func=None,
        n_layers=5,
        ns=200,
        depth=9,
        p=0.25,
        mass_cons_updates=True,
        loss_absolute=True,
        mass_cons_moments=True,
        hard_constraints_updates=True,
        hard_constraints_moments=False,
        multi_step=False,
        step_size=1,
        moment_scheme=2,
        plot_while_training=False,
        plot_all_moments=True,
        use_batch_norm=False,
        use_dropout=False,
        single_sim_num=None,
        avg_dataloader=False,
        pretrained_path=None
    ):
        super().__init__()
        self.moment_scheme = moment_scheme
        self.out_features = moment_scheme * 2
        self.lr = learning_rate
        self.loss_func = loss_func
        self.beta = beta
        self.batch_size = batch_size

        self.loss_absolute = loss_absolute

        """ Using the following only for multi-step training"""
        self.hard_constraints_updates = hard_constraints_updates
        self.hard_constraints_moments = hard_constraints_moments
        self.mass_cons_updates = mass_cons_updates  # Serves no purpose now
        self.mass_cons_moments = mass_cons_moments

        self.multi_step = multi_step
        self.step_size = step_size
        self.num_workers = 48
        self.plot_while_training = plot_while_training
        self.plot_all_moments = plot_all_moments
        self.single_sim_num = single_sim_num
        self.avg_dataloader = avg_dataloader
        self.save_hyperparameters()

        self.updates_std = updates_std
        self.updates_mean = updates_mean
        self.inputs_mean = inputs_mean
        self.inputs_std = inputs_std

        # Some plotting stuff
        self.color = ["#26235b", "#bc473a", "#812878", "#f69824"]
        self.var = ["Lc", "Nc", "Lr", "Nr"]
        self.pretrained_path = pretrained_path
        self.model = self.initialization_model(
            act,
            n_layers,
            ns,
            self.out_features,
            depth,
            p,
            use_batch_norm,
            use_dropout,
            save_dir,
            pretrained_path,
        )

    @staticmethod
    def initialization_model(
        act,
        n_layers,
        ns,
        out_features,
        depth,
        p,
        use_batch_norm,
        use_dropout,
        save_dir,
        pretrained_path,
    ):
        model = plNetwork(
            act, n_layers, ns, out_features, depth, p, use_batch_norm, use_dropout
        )
        model.train()
        return model

    def forward(self):
        self.updates = self.model(self.x)
        self.norm_obj = normalizer(
            self.updates,
            self.x,
            self.y,
            self.updates_mean,
            self.updates_std,
            self.inputs_mean,
            self.inputs_std,
            self.device,
            self.hard_constraints_updates,
            self.hard_constraints_moments,
        )
        (
            self.real_x,
            self.real_y,
            self.pred_moment,
            self.pred_moment_norm,
        ) = self.norm_obj.calc_preds()
        self.pred_moment, self.pred_moment_norm = self.norm_obj.set_constraints()

    def test_step(self, initial_moments):

        """For moment-wise evaluation as used for ODE solve"""
        with torch.no_grad():
            preds = self.model(initial_moments.float())
        return preds



In [5]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

    
class simulation_forecast:
    def __init__(
        self,
        all_moments_in,
        new_model,
        inputs_mean,
        inputs_std,
        updates_mean,
        updates_std,
        device
        
    ):
        self.all_moments_in = all_moments_in
        """ Assuming the moments to be arranged 
        as Lc, Nc,Lr, Nr """
            
        self.inputs_mean = inputs_mean
        self.inputs_std = inputs_std
        self.updates_mean = updates_mean
        self.updates_std = updates_std
        self.model = new_model
        #Since moments_out is returned to main, it should be numpy at the end of the solver call
        self.moments_out = None
        self.device = device
        #self.lo = all_moments_in[0]+ all_moments_in[2]
        
        self.setup()
        self.create_input()
        self.model.eval()
        

    def setup(self):
        #self.all_moments_in = self.all_moments_in.astype(np.float32)
        self.all_moments_in = self.all_moments_in.reshape(-1,4) #Now we can process the whole batch
        #self.all_moments_in = torch.from_numpy(self.all_moments_in).float().to(self.device)
        
    def test(self):
        
        predictions_updates = self.model.test_step(self.inputs)
        self.moment_calc(predictions_updates)

        #self.moments_out = self.moments_out.astype(np.float64)

    # For Calculation of Moments
    def calc_mean(self, no_norm, means, stds):
        return (no_norm - means.reshape(-1,)) / stds.reshape(
            -1,
        )

    # For creation of inputs
    def create_input(self):
        divisor = (self.all_moments_in[:,2] + self.all_moments_in[:,0]).cpu()
        tau = np.divide( self.all_moments_in[:,2].cpu(), divisor, where=divisor!=0 ).to(self.device)

        xc = (self.all_moments_in[:,0] / (self.all_moments_in[:,1] + 1e-8)).to(self.device)
        self.lo_arr = (self.all_moments_in[:,2] + self.all_moments_in[:,0]).to(self.device)
        self.rm = torch.from_numpy(np.full((self.lo_arr.shape),13e-6)).to(self.device)
        self.nu = torch.from_numpy(np.full((self.lo_arr.shape),1)).to(self.device)

        self.model_params = torch.cat((self.lo_arr.reshape(-1,1), 
                                            self.rm.reshape(-1,1),self.nu.reshape(-1,1)),
                                           axis=1)
        
        #self.model_params = np.asarray([self.lo,self.rm,self.nu])
        inputs = torch.cat(
            (
                self.all_moments_in,
                tau.reshape(-1, 1),
                xc.reshape(-1, 1),
                self.model_params
            ),
            axis=1,
        )
        self.inputs = self.calc_mean(inputs, self.inputs_mean, self.inputs_std)

    def check_preds(self):
        self.moments_out[:, 0] = (self.lo_arr) - self.moments_out[:, 2]
        self.moments_out[self.moments_out<0] = 0
    
    def moment_calc(self, predictions_updates):
        self.updates = (
            predictions_updates.detach() * self.updates_std
        ) + self.updates_mean
        
        self.moments_out = (self.all_moments_in[:,0:4] + (self.updates[:,:] * 20))
        self.check_preds()
        

In [6]:
inputs_mean = torch.from_numpy(np.asarray([[0.0002621447787797809, 51128093.51524663,
                0.0003302890736022656, 5194.251154308974,
                0.5566250557023539, 4.8690682855354596e-12,
                0.0005924338523807814, 1.0848856769219835e-05,
                2.0193905073168525]])).to('cuda')

inputs_std = torch.from_numpy(np.asarray([[0.0003865559774857862, 86503916.13808665,
                0.00041369562655559327, 19127.947970150628,
                0.46107363560819126, 3.873092422358367e-12,
                0.00042887039563850967, 1.920461805101116e-06,
                1.3098055608321857]])).to('cuda')

updates_mean = torch.from_numpy(np.asarray([[-8.527820407019667e-08, -13961.459867976775,
                8.527678028525988e-08, 0.010221931180955181]])).to('cuda')

updates_std = torch.from_numpy(np.asarray([[3.600841676033818e-07, 55095.904252313965,
                3.6008419243808887e-07, 68.6678997504877]])).to('cuda')

    
pl_model = LightningModel(inputs_mean=inputs_mean, inputs_std=inputs_std,
                            updates_mean=updates_mean, updates_std=updates_std) 

model_path = '/work/ka1176/caroline/gitlab/icon-aes/externals/mlbridges/cffi_interface/trained_models/best_model.ckpt'
trained_model = pl_model.load_from_checkpoint(model_path).to('cuda')


In [7]:
shape = (4, 1, 44)
current_moments = np.zeros(shape)
current_moments[0] = 5.8e-4
current_moments[1] = 3.4e7
current_moments[2] = 3.89e-4
swapped_moments = np.swapaxes(current_moments,0, 2).reshape(-1, 4)

swapped_moments = torch.from_numpy(swapped_moments).to('cuda')

In [13]:
%%time
new_forecast = simulation_forecast(swapped_moments, trained_model,
                                   inputs_mean, inputs_std,
                                   updates_mean, updates_std,
                                   'cuda'
                                  )

CPU times: user 1.26 ms, sys: 8.47 ms, total: 9.73 ms
Wall time: 116 ms


In [28]:
gpu_t = %timeit -o -n 200 -r 200 new_forecast.test()

267 µs ± 34.3 µs per loop (mean ± std. dev. of 200 runs, 200 loops each)


In [29]:
t_gpu_median = np.median(gpu_t.timings)
t_gpu = np.mean(gpu_t.timings)

In [30]:
t_cpu = 7.921824e-04 

In [31]:
print(t_gpu)
print(t_gpu_median)
print(t_cpu)

0.0002666758746501728
0.0002631261774877203
0.0007921824


In [32]:
t_gpu / t_cpu

0.3366344350116498